# Notebook para leer los txt

## 1. Raw Data Folder

* 30 participantes (14 sanos y 16 pacientes)
* 9 ejercicios
* x6 repeticiones
* 30 fps
* 2598 videos

In [8]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import os # para interactur con el sistema operativo

### 1.1 Datos de los nombres de los archivos

SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt

* SubjectID: id uniquely identifying the person performing the exercise
* DateID: id identifying the session in which the person was recorded
* GestureLabel: Label identifying the gesture; possible values are from 0 to 8
* RepetitionNumber: Each gesture was repeated several times and this shows the repetition number
* CorrectLabel: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* Position: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

In [9]:
# función para guardar en el dataframe los datos que aparecen en los nombre de los archivos
def leer_nombre_archivo(archivo:str):
    """
    Obtiene los datos de interés del nombre de los archivos

    :param archivo: nombre del archivo (SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt)
    :return: devuelve una lista de strings con los datos del nombre del archivo
    (SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position)
    """ 
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [10]:
def leer_datos_archivo(directorio:str, columnas:list[str]):

    """
    Compila toda la información de los archivos de un directorio y los guarda en un dataframe

    :param directorio: Nombre del directirio donde están los archivos
    :param columnas: Lista con el nombre de las columnas que queremos que tenga el dataframe de salida
    :return: DataFrame con todos los datos recompilados
    """
    file_list = os.listdir(directorio) # crea una lista con los nombre de los arhivos que se encuentran en ese directorio

    # Initialize an empty list to store the extracted data
    list_data = []

    # Iterate through each raw data file
    for file_name in file_list:
        campos = leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                # Split the line by comma and extract the required information
                line_data = line.strip().split(',')[3:]  # Skip the initial timestamp and other unwanted data
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] # quitar los paréntesis
                for i in range(0, len(cleaned_data), 7): # por cada linea de los archivos necesitamos bloques de 7 valores
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df


In [11]:
directory = '../dataset/SkeletonData/RawData' # directorio donde se encuentran los datos

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

df_data = leer_datos_archivo(directory, columnas)


In [12]:
# se va a añadir una columna para que guarde el número de frame
df_data['FrameNumber'] = (df_data.index // 25) + 1

In [13]:
df_data.to_csv('../csvFiles/raw_pacientes.csv', index=False) # lo guardamos a cvs

In [15]:
df_data.head(50)

,SubjectID,DateID,GestureLabel,RepetitionNumber,CorrectLabel,Position,JointName,TrackedStatus,3D_X,3D_Y,3D_Z,2D_X,2D_Y,FrameNumber
0,101,18,0,1,1,stand,SpineBase,Tracked,-0.1028086,0.06965441,2.464606,243.133,196.5854,1
1,101,18,0,1,1,stand,SpineMid,Tracked,-0.1026228,0.3837799,2.438919,242.971,149.1077,1
2,101,18,0,1,1,stand,Neck,Tracked,-0.1025293,0.6877351,2.40196,242.6967,101.3569,1
3,101,18,0,1,1,stand,Head,Tracked,-0.1190992,0.8358598,2.373549,239.9086,76.90204,1
4,101,18,0,1,1,stand,ShoulderLeft,Tracked,-0.2826451,0.5525576,2.38421,214.7496,121.544,1
5,101,18,0,1,1,stand,ElbowLeft,Tracked,-0.3382697,0.2775677,2.420022,207.0312,164.7725,1
6,101,18,0,1,1,stand,WristLeft,Tracked,-0.375412,0.05293661,2.374334,200.3178,198.7601,1
7,101,18,0,1,1,stand,HandLeft,Tracked,-0.3878586,-0.006873231,2.360761,198.0391,208.0262,1
8,101,18,0,1,1,stand,ShoulderRight,Tracked,0.07524497,0.553144,2.399761,269.9869,122.0691,1
9,101,18,0,1,1,stand,ElbowRight,Tracked,0.1186013,0.2708487,2.436513,276.3187,166.1256,1
